In [37]:
import pandas as pd
import numpy as np
import json
import os
from re_ocr_resource.config import txt_files_path

def load_json(x):
    try:
        return json.loads(x)
    except:
        return x
    
    
books = txt_files_path.glob("*/")

urns = ["URN:NBN:" + os.path.basename(book) for book in books]

In [5]:
len(urns)

24259

In [46]:
df = pd.read_csv("results.csv", index_col=0)
errors = df.loc[df.error.notna()]
df = df.loc[df.error.isna()]

In [83]:
errors["error"].values

array(['Could not find urn URN:NBN:no-nb_digibok_2021042148873',
       'Could not find urn URN:NBN:no-nb_digibok_2013050224022',
       'Could not find urn URN:NBN:no-nb_digibok_2017032448026',
       'Could not find urn URN:NBN:no-nb_digibok_2020113026009',
       'Could not find urn URN:NBN:no-nb_digibok_2018062548005'],
      dtype=object)

In [11]:
with open(file="urns_all_info.json") as f:
    urns_all_info = json.load(f)

In [57]:
urns_all_info[1]

{'id': 'URN:NBN:no-nb_digibok_2010020903013',
 'accessInfo': {'accessAllowedFrom': 'EVERYWHERE',
  'viewability': 'ALL',
  'license': 'publicdomain',
  'actions': ['download'],
  'accessInfoLastModified': '2018-10-09T08:46:36.729Z',
  'isDigital': True,
  'isPublicDomain': True}}

In [65]:
lst = []
for row in urns_all_info:
    if "error" in row:
        continue
    try:
        dct = row["accessInfo"]
        urn = row["id"]
        pb = dct["isPublicDomain"]
    
        lic = dct.get("license", np.NAN)
        actions = dct.get("actions", np.NAN)
        
        lst.append({"urn": urn, "public_domain": pb, "license": lic, "actions": actions})
    except:
        print(row)

In [67]:
df = pd.DataFrame(lst)

In [71]:
df.columns

Index(['urn', 'public_domain', 'license', 'actions'], dtype='object')

In [73]:
df.value_counts("public_domain")

public_domain
True     19060
False     5194
Name: count, dtype: int64

In [77]:
df.value_counts("public_domain", dropna=False), df.license.value_counts(dropna=False), df.actions.value_counts(dropna=False)

(public_domain
 True     19060
 False     5194
 Name: count, dtype: int64,
 license
 publicdomain    19060
 copyrighted      2876
 bokhylla         2273
 ccbyncnd           45
 Name: count, dtype: int64,
 actions
 [download]    20922
 NaN            3332
 Name: count, dtype: int64)

In [80]:
df.loc[df.public_domain == True].license.value_counts(dropna=False)

license
publicdomain    19060
Name: count, dtype: int64

In [78]:
df.to_csv("re_ocr_public_domain.csv")